# ipygany with proteus

In [ ]:
from ipydatawidgets import NDArrayWidget

from ipywidgets import Play, IntProgress, link, HBox

from pythreejs import BoxGeometry, Mesh, MeshStandardMaterial, Side

from ipygany import Scene, TetraMesh, Alpha, Threshold
from ipygany.vtk_loader import load_vtk, get_ugrid_data

In [ ]:
box = Mesh(
    geometry=BoxGeometry(width=0.16, height=0.4, depth=0.16), 
    material=MeshStandardMaterial(color='lightgreen'),
    position=(2.47, 0.5, 0.08),
)

boundaries = Mesh(
    geometry=BoxGeometry(width=3.22, height=1., depth=1.), 
    material=MeshStandardMaterial(color='lightgray', side=Side.BackSide),
    position=(1.61, 0.5, 0.5),
)

In [ ]:
# Caching arrays on the front-end using NDArrayWidgets
phi = []
p = []
for i in range(77):
    grid = load_vtk('proteus_vtu/proteus_{}.vtu'.format(i))
    grid_data = get_ugrid_data(grid)
    
    phi_array = grid_data['phi']['X1']['array']
    p_array = grid_data['p']['X1']['array']

    phi.append(NDArrayWidget(array=phi_array))
    p.append(NDArrayWidget(array=p_array))

In [ ]:
mesh = TetraMesh.from_vtk('proteus_vtu/proteus_1.vtu', environment_meshes=[box, boundaries])

phi_component = mesh['phi', 0]
p_component = mesh['p', 0]

def update_step(change):
    step = change['new']
    phi_component.array = phi[step]
    p_component.array = p[step]

play = Play(description='Step:', min=0, max=76, value=0, interval=40)
play.observe(update_step, names=['value'])

progress = IntProgress(value=0, step=1, min=0, max=76)
link((progress, 'value'), (play, 'value'))

threshold = Threshold(mesh, input='phi', min=-100, max=0, dynamic=True)
wave = Alpha(threshold)

stepper = HBox((play, progress))
stepper

In [ ]:
scene = Scene((wave, ))

scene